In [1]:
## Python Import Statements
import os
import pandas as pd
import numpy as np
import scipy as sp
from scipy import stats
from scipy import optimize
import matplotlib.pyplot as plt

ROOTDIR = os.getcwd() # Home directory
CODEDIR = ROOTDIR + '/CODE/' # Code directory
DATADIR = ROOTDIR + '/DATA/' # Data directory

os.chdir(CODEDIR) # Change directory
import changeTimes
import getRunningMean
import conditional_copula_ts
import iso_sine_signal
import calcSiteStats

os.chdir(ROOTDIR) # Change directory

In [2]:
# User defined variables
sitenames = ['DELA']
ensemble_size = 10

In [5]:
for s in np.arange(len(sitenames)):
    # 30min Precipitation Data
    df_P30 = pd.read_excel(DATADIR+'PrecipData/'+str(sitenames[s])+'PrecipData.xlsx',index=False)
    df_P30 = changeTimes.change_Pdata(df_P30)
    
    # Sum to Daily Precipitation Amounts
    #checks for primary precip data
    if 'priPrecipBulk' in df_P30:
        df_P30b = df_P30.set_index('DateTime')
        precip_daily = df_P30b['priPrecipBulk'].resample('D').sum() # sum total daily P
        frac_year = df_P30b['FracYear'].resample('D').mean() # average daily FracYear (min or max better?)

        daily_P = pd.DataFrame({'Total P':precip_daily,'FracYear':frac_year})
        daily_P['Total P'].replace(0, np.nan, inplace=True)


        # Biweekly Stable Water Isotope Data
        df_iso = pd.read_excel(DATADIR+'IsoData/'+str(sitenames[s])+'IsoData.xlsx',index=False)
        df_iso = changeTimes.change_ISOdata(df_iso)



        # Create Biweekly Precipitation Amount Timeseries to Correspond to Recorded Isotope Values
        df_iso['setDate'] = pd.to_datetime(df_iso['setDate'])  
        df_iso['collectDate'] = pd.to_datetime(df_iso['collectDate']) 

        P14 = []
        for i in np.arange(len(df_iso['setDate'])):
            subset = ((df_P30['DateTime'] > df_iso['setDate'].iloc[i]) 
                    & (df_P30['DateTime'] <= df_iso['collectDate'].iloc[i]))

            df_sub = df_P30.loc[subset]
            P14.append(np.nansum(df_sub['priPrecipBulk'].values))

        df_iso['Total P'] = P14
        del P14, i, subset
        
    else:
        df_P30b = df_P30.set_index('DateTime')
        precip_daily = df_P30b['secPrecipBulk'].resample('D').sum() # sum total daily P
        frac_year = df_P30b['FracYear'].resample('D').mean() # average daily FracYear (min or max better?)

        daily_P = pd.DataFrame({'Total P':precip_daily,'FracYear':frac_year})
        daily_P['Total P'].replace(0, np.nan, inplace=True)


        # Biweekly Stable Water Isotope Data
        df_iso = pd.read_excel(DATADIR+'IsoData/'+str(sitenames[s])+'IsoData.xlsx',index=False)
        df_iso = changeTimes.change_ISOdata(df_iso)



        # Create Biweekly Precipitation Amount Timeseries to Correspond to Recorded Isotope Values
        df_iso['setDate'] = pd.to_datetime(df_iso['setDate'])  
        df_iso['collectDate'] = pd.to_datetime(df_iso['collectDate']) 

        P14 = []
        for i in np.arange(len(df_iso['setDate'])):
            subset = ((df_P30['DateTime'] > df_iso['setDate'].iloc[i]) 
                    & (df_P30['DateTime'] <= df_iso['collectDate'].iloc[i]))

            df_sub = df_P30.loc[subset]
            P14.append(np.nansum(df_sub['secPrecipBulk'].values))

        df_iso['Total P'] = P14
        del P14, i, subset